In [1]:

# Set up notebook
%pprint
import sys
if ('../py' not in sys.path): sys.path.insert(1, '../py')
from FRVRS import nu, fu
nu.delete_ipynb_checkpoint_folders()

Pretty printing has been turned OFF


In [11]:

data_frames_dict = nu.load_data_frames(
    metrics_evaluation_open_world_df='', metrics_evaluation_open_world_file_stats_df=''
)
logs_df = data_frames_dict['metrics_evaluation_open_world_df']
csv_files_list = sorted(logs_df.csv_file_name.unique())
file_stats_df = data_frames_dict['metrics_evaluation_open_world_file_stats_df']
json_files_list = sorted(file_stats_df.json_file_name.unique())

Attempting to load /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/pkl/metrics_evaluation_open_world_df.pkl.
Attempting to load /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/pkl/metrics_evaluation_open_world_file_stats_df.pkl.


In [18]:

Human_Sim_Metrics_csv_files_set = set()
Human_Sim_Metrics_json_files_set = set()
folder_path = '../data/logs/Human_Sim_Metrics_Data 4-12-2024'
for sub_directory, directories_list, files_list in os.walk(folder_path):
    for file_name in files_list:
        if file_name.endswith('.json'):
            # file_path = osp.join(sub_directory, file_name)
            Human_Sim_Metrics_json_files_set.add(file_name)
        elif file_name.endswith('.csv'):
            # file_path = osp.join(sub_directory, file_name)
            Human_Sim_Metrics_csv_files_set.add(file_name)

In [19]:

set(csv_files_list).difference(Human_Sim_Metrics_csv_files_set)

set()

In [20]:

Human_Sim_Metrics_csv_files_set.difference(set(csv_files_list))

{'703b9d0b-7786-4a4b-8a26-7c57d16b785a_2024226.csv', 'a3a7727b-dd25-4e78-8d63-e66573f5077e_2024224.csv', '84c92e21-764b-435b-9e55-0f12f60383fc_2024227.csv', 'a9c21d35-e8be-47fd-926a-a00dcd3e792b_2024225.csv', '8b6c4aed-0983-4a76-99fb-cec08dc26b92_2024224.csv', '05e0be07-479a-4ea4-9b90-3fc7b79ef7ff_2024225.csv', '9f3bb117-3a4b-4bc7-9237-e2d9920d5333_2024227.csv', '0e8693a1-1926-4d56-b0c4-ee2dc8181fc6_2024226.csv'}

In [21]:

set(json_files_list).difference(Human_Sim_Metrics_json_files_set)

set()

In [22]:

Human_Sim_Metrics_json_files_set.difference(set(json_files_list))

{'a3a7727b-dd25-4e78-8d63-e66573f5077e_2024224.json', '703b9d0b-7786-4a4b-8a26-7c57d16b785a_2024226.json', 'a9c21d35-e8be-47fd-926a-a00dcd3e792b_2024225.json', '0e8693a1-1926-4d56-b0c4-ee2dc8181fc6_2024226.json', '8b6c4aed-0983-4a76-99fb-cec08dc26b92_2024224.json', '84c92e21-764b-435b-9e55-0f12f60383fc_2024227.json', '05e0be07-479a-4ea4-9b90-3fc7b79ef7ff_2024225.json', '9f3bb117-3a4b-4bc7-9237-e2d9920d5333_2024227.json'}

In [23]:

Open_World_Metrics_csv_files_set = set()
Open_World_Metrics_json_files_set = set()
folder_path = '../data/logs/Metrics Evaluation Open World'
for sub_directory, directories_list, files_list in os.walk(folder_path):
    for file_name in files_list:
        if file_name.endswith('.json'):
            # file_path = osp.join(sub_directory, file_name)
            Open_World_Metrics_json_files_set.add(file_name)
        elif file_name.endswith('.csv'):
            # file_path = osp.join(sub_directory, file_name)
            Open_World_Metrics_csv_files_set.add(file_name)

In [24]:

set(csv_files_list).difference(Open_World_Metrics_csv_files_set)

set()

In [25]:

Open_World_Metrics_csv_files_set.difference(set(csv_files_list))

{'703b9d0b-7786-4a4b-8a26-7c57d16b785a_2024226.csv', 'a3a7727b-dd25-4e78-8d63-e66573f5077e_2024224.csv', '84c92e21-764b-435b-9e55-0f12f60383fc_2024227.csv', 'a9c21d35-e8be-47fd-926a-a00dcd3e792b_2024225.csv', '8b6c4aed-0983-4a76-99fb-cec08dc26b92_2024224.csv', '05e0be07-479a-4ea4-9b90-3fc7b79ef7ff_2024225.csv', '9f3bb117-3a4b-4bc7-9237-e2d9920d5333_2024227.csv', '0e8693a1-1926-4d56-b0c4-ee2dc8181fc6_2024226.csv'}

In [27]:

set(json_files_list).difference(Open_World_Metrics_json_files_set)

set()

In [28]:

Open_World_Metrics_json_files_set.difference(set(json_files_list))

{'a3a7727b-dd25-4e78-8d63-e66573f5077e_2024224.json', '703b9d0b-7786-4a4b-8a26-7c57d16b785a_2024226.json', 'a9c21d35-e8be-47fd-926a-a00dcd3e792b_2024225.json', '0e8693a1-1926-4d56-b0c4-ee2dc8181fc6_2024226.json', '8b6c4aed-0983-4a76-99fb-cec08dc26b92_2024224.json', '84c92e21-764b-435b-9e55-0f12f60383fc_2024227.json', '05e0be07-479a-4ea4-9b90-3fc7b79ef7ff_2024225.json', '9f3bb117-3a4b-4bc7-9237-e2d9920d5333_2024227.json'}

In [29]:

folder_path = '../data/logs/Metrics Evaluation Open World'
for sub_directory, directories_list, files_list in os.walk(folder_path):
    for file_name in files_list:
        if file_name in Open_World_Metrics_json_files_set:
            file_path = osp.join(sub_directory, file_name)
            print(file_path)
        if file_name in Open_World_Metrics_csv_files_set:
            file_path = osp.join(sub_directory, file_name)
            print(file_path)

../data/logs/Metrics Evaluation Open World/02d472ac-f6fe-474c-815d-6125fadfcbf7_2024211/02d472ac-f6fe-474c-815d-6125fadfcbf7_2024211.csv
../data/logs/Metrics Evaluation Open World/02d472ac-f6fe-474c-815d-6125fadfcbf7_2024211/02d472ac-f6fe-474c-815d-6125fadfcbf7_2024211.json
../data/logs/Metrics Evaluation Open World/04f80090-9e61-431d-8473-dccb75fed04d_2024207/04f80090-9e61-431d-8473-dccb75fed04d_2024207.csv
../data/logs/Metrics Evaluation Open World/04f80090-9e61-431d-8473-dccb75fed04d_2024207/04f80090-9e61-431d-8473-dccb75fed04d_2024207.json
../data/logs/Metrics Evaluation Open World/05e0be07-479a-4ea4-9b90-3fc7b79ef7ff_2024225/05e0be07-479a-4ea4-9b90-3fc7b79ef7ff_2024225.csv
../data/logs/Metrics Evaluation Open World/05e0be07-479a-4ea4-9b90-3fc7b79ef7ff_2024225/05e0be07-479a-4ea4-9b90-3fc7b79ef7ff_2024225.json
../data/logs/Metrics Evaluation Open World/0bc62b93-ac4d-40ef-bdfe-1d7badb24f70_2024208/0bc62b93-ac4d-40ef-bdfe-1d7badb24f70_2024208.csv
../data/logs/Metrics Evaluation Open W

In [2]:

# load libraries
from datetime import timedelta
from pandas import DataFrame
import humanize
import matplotlib.pyplot as plt
import numpy as np
import os
import os.path as osp
import pandas as pd
import re

In [3]:

print([f'os.{fn}' for fn in dir(os) if 'dir' in fn.lower()])
print([f'osp.{fn}' for fn in dir(osp) if 'dir' in fn.lower()])
print([f'nu.{fn}' for fn in dir(nu) if 'github' in fn])

['os.DirEntry', 'os.O_DIRECT', 'os.O_DIRECTORY', 'os.ST_NODIRATIME', 'os.chdir', 'os.curdir', 'os.fchdir', 'os.listdir', 'os.makedirs', 'os.mkdir', 'os.pardir', 'os.removedirs', 'os.rmdir', 'os.scandir', 'os.supports_dir_fd']
['osp.curdir', 'osp.dirname', 'osp.isdir', 'osp.pardir']
['nu.github_folder']


In [4]:

print([f'os.{fn}' for fn in dir(os) if 'name' in fn.lower()])
print([f'osp.{fn}' for fn in dir(osp) if 'name' in fn.lower()])
print([f'nu.{fn}' for fn in dir(nu) if 'name' in fn])

['os.__name__', 'os.confstr_names', 'os.name', 'os.pathconf_names', 'os.rename', 'os.renames', 'os.sysconf_names', 'os.ttyname', 'os.uname', 'os.uname_result']
['osp.__name__', 'osp.basename', 'osp.dirname', 'osp.supports_unicode_filenames']
['nu.get_filename_from_url']


In [5]:

os.listdir(path=None)

['Attic.ipynb', 'Installs.ipynb', 'OS Path Navigation.ipynb']

In [16]:

for sub_directory, directories_list, files_list in os.walk(nu.github_folder):
    file_names_list = []
    for file_name in files_list:
        if file_name.endswith('.ipynb') and re.search(r'(\bOW\b|Open(_|%20| )World)', file_name):
            file_names_list.append(file_name)
    if file_names_list:
        print()
        print(sub_directory.split('/')[-1])
        print('\n'.join(file_names_list))


open_world
Analyze OW Patient Stats.ipynb
Correct SORT Order Metric for Metrics Evaluation Open World.ipynb
Data Fixes for Metrics Evaluation Open World.ipynb
Dataset Built for Metrics Evaluation Open World.ipynb
Examine Freeform Open World ADM JSON Files.ipynb
Exploratory Analysis of Open World ITM Data.ipynb
Explore OW Participants and Scenes.ipynb
File Stats Created for Metrics Evaluation Open World.ipynb
Scene Stats Created for Metrics Evaluation Open World.ipynb


In [ ]:

# Define the starting directory
root_dir = '/'

# Use a recursive function to search for matching folders
def find_python311_folders(directory):
    found_folders = []
    try: entries_list = os.scandir(directory)
    except: entries_list = []
    for entry in entries_list:
        try: is_entry_dir = entry.is_dir()
        except: is_entry_dir = False
        if is_entry_dir:
            
            # Check if the current entry is the target folder
            if entry.name == "python3.11": found_folders.append(os.path.join(directory, entry.name))
            
            # Recursively search subdirectories (except the target folder)
            elif entry.name != "python3.11": found_folders.extend(find_python311_folders(os.path.join(directory, entry.name)))
    
    return found_folders

# Start the search from the root directory
python311_folders = find_python311_folders(root_dir)

In [ ]:

# Print the found folders
print(f"Found python3.11 folders: {', '.join(python311_folders)}")

In [ ]:

# Print out a section of the .gitignore
black_list = ['.ipynb_checkpoints', '$Recycle.Bin', '.git']
print()
print('# exclude everything except personal directories')
print('/*')
print('!.gitignore')
for sub_directory in os.listdir(nu.github_folder):
    if all(map(lambda x: x not in sub_directory, black_list)) and osp.isdir(osp.join(nu.github_folder, sub_directory)): print(f'!/{sub_directory}')
print('!README.md')

In [ ]:

columns_set = set()
for pickle_name in [fn.split('.')[0] for fn in os.listdir(s.saves_pickle_folder)]:
    if pickle_name.endswith('_df'):
        df = nu.load_object(pickle_name)
        columns_set.update(df.columns[list(map(lambda x: ('year' in x) and (('begin' in x) or ('end' in x)), df.columns))])
columns_set

In [ ]:

# 'Intel64 Family 6 Model 60 Stepping 3, GenuineIntel'
import platform

platform.processor()